In [1]:
print("Hello")

Hello


In [2]:
import json
import time
import requests
import os
import re

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
#  r = requests.get(url)
#  soup = BeautifulSoup(r.text, "html.parser")

In [3]:
# using a cookies.json file
def load_cookie_dict(file):
  with open(file, "r") as f:
    cookies_list = json.load(f)

    cookies_dict = {}
    for cookie in cookies_list:
      cookies_dict[cookie['name']] = cookie['value']
    return cookies_dict  # Return the dictionary

In [4]:
# using a cookies.json file
def load_cookies_list(file):
  with open(file, "r") as f:
    cookies_list = json.load(f)

    cookies_list = [(cookie["name"], cookie["value"]) for cookie in cookies_list]
    return cookies_list  # Return the dictionary

In [22]:
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")

# cookies = load_cookies_list("canvas-cookies.json")
cookie_file = "canvas-cookies.json" 
with open(cookie_file, "r") as f:
  cookies = json.load(f)
request_cookies = load_cookie_dict(cookie_file)

cookie_file = "video-cookies.json" 
with open(cookie_file, "r") as f:
  video_cookies = json.load(f)

# print(cookies)
# print(video_cookies)

In [6]:
driver = webdriver.Chrome(options=chrome_options)

for cookie in cookies:
  driver.execute_cdp_cmd('Network.setCookie', cookie)

# driver.implicitly_wait(2)
start_url = "https://canvas.ucdavis.edu/courses/961262/files"
driver.get(start_url)

# driver.refresh()
# get_url = driver.current_url # Check if redirected ? Some courses have blocked file directory

# print(driver.page_source.encode("utf-8"))
# print(driver.get_cookies())

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "ef-item-row"))
)
print("Page is ready!")

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body"))

files = []
for row in soup.find_all("div", class_="ef-item-row"):
  anchor = row.find("a")
  link = anchor.get("href")
  name = anchor.find("span", class_="ef-name-col__text").text.strip()
  size = row.find("div", class_="ef-size-col").text.strip()

  # not_right_side
  files.append((link, name, size))
print(files)
print(len(files))


driver.quit()

Page is ready!
[('https://canvas.ucdavis.edu/files/26826815/download?download_frd=1', 'Aphasia.Short.problem house model.pdf', '628 KB'), ('https://canvas.ucdavis.edu/files/26968586/download?download_frd=1', 'ASL and Brain Ling 175.2025.pdf', '2.8 MB'), ('https://canvas.ucdavis.edu/files/26882167/download?download_frd=1', 'BILINGUALISM CHAPTER 6.2025.pdf', '1.6 MB'), ('https://canvas.ucdavis.edu/files/26437251/download?download_frd=1', 'Brain in Space.2025.pdf', '2.4 MB'), ('https://canvas.ucdavis.edu/files/26346274/download?download_frd=1', 'Brain in Time 2025.pdf', '1.3 MB'), ('https://canvas.ucdavis.edu/files/26799137/download?download_frd=1', 'Brain Models and Aphasia 2025.pdf', '2.2 MB'), ('https://canvas.ucdavis.edu/files/26706226/download?download_frd=1', 'Brain.Models 2025.pdf', '1.3 MB'), ('https://canvas.ucdavis.edu/files/26684972/download?download_frd=1', 'Double Dissociation.1.30.2025.pptx', '4.5 MB'), ('/courses/961262/files/folder/Exam%20Materials', 'Exam Materials', '--'

In [7]:
foldername = "course-downloads"
os.makedirs(foldername, exist_ok=True)


url = "https://canvas.ucdavis.edu/files/26826815/download?download_frd=1"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url,  headers=headers, cookies=request_cookies)
with open("course-downloads/example_file", mode="wb") as f:
  f.write(response.content)

```
driver = webdriver.Chrome(options=chrome_options)

for cookie in cookies:
  driver.execute_cdp_cmd('Network.setCookie', cookie)

start_url = "https://canvas.ucdavis.edu/courses/961262/external_tools/5280"
driver.get(start_url)


tool_content_pattern = re.compile("^tool_content_\d{3}")
# element = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.NAME, tool_content_pattern))
# )
# print("Page is ready!")

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body"))


iframe = soup.find(id=tool_content_pattern)
iframe_name = iframe.get("name")

driver.switch_to.frame(driver.find_element(by=By.NAME, value=iframe_name))
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body").prettify())


# the following links are all relative to aggievideo.canvas.ucdavis.edu
media_links = set()
for anchor in soup.find_all("a", attrs={"class": "item_link"}):
  link = anchor.get("href")
  media_links.add(link)
print(media_links)
print(len(media_links))

driver.quit()
```

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

for cookie in cookies:
  driver.execute_cdp_cmd('Network.setCookie', cookie)

start_url = "https://canvas.ucdavis.edu/courses/961262/external_tools/5280"
driver.get(start_url)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body").prettify())

time.sleep(2)

tool_content_pattern = re.compile("^tool_content_\d{3}")
iframe = soup.find(id=tool_content_pattern)
if not iframe:
  print("not found")
  driver.quit()
  exit()
else: 
  print("Iframe found, continuing")
iframe_name = iframe.get("name")

driver.switch_to.frame(driver.find_element(by=By.NAME, value=iframe_name))
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body").prettify())
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "channelGallery_scroller_alert")))
time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body").prettify())


# the following links are all relative to aggievideo.canvas.ucdavis.edu
media_links = set()
for anchor in soup.find_all("a", attrs={"class": "item_link"}):
  link = anchor.get("href")
  media_links.add(link)
# print(media_links)
print(f"Found: {len(media_links)}")
print("If only 15 found, rerun until content loads")

driver.quit()


Iframe found, continuing
{'/media/LIN-175%3A+2025-02-13+10%3A27/1_5p0hnlmo/365313982', '/media/LIN-175%3A+2025-02-20+10%3A27/1_rat5no92/365313982', '/media/Clip+of+LIN-175%3A+2025-02-11+10%3A27/1_2sfwuc5u/365313982', '/media/LIN-175%3A+2025-01-07+10%3A27/1_oqw8q1fx/365313982', '/media/LIN-175%3A+2025-02-27+10%3A27/1_yf4mwdee/365313982', '/media/LIN-175%3A+2025-01-30+10%3A27/1_c8z8uzp7/365313982', '/media/LIN-175%3A+2025-03-13+10%3A27/1_8025ziy9/365313982', '/media/LIN-175%3A+2025-02-04+10%3A27/1_q71p4dhf/365313982', '/media/LIN-175%3A+2025-03-06+10%3A27/1_mbz6ul4h/365313982', '/media/LIN-175%3A+2025-01-28+10%3A27/1_nmu6gd5l/365313982', '/media/LIN-175%3A+2025-01-09+10%3A27/1_m53bly5a/365313982', '/media/LIN-175%3A+2025-01-21+10%3A27/1_li507l6w/365313982', '/media/LIN-175%3A+2025-02-25+10%3A27/1_o732eocy/365313982', '/media/LIN-175%3A+2025-01-23+10%3A27/1_88yxizrs/365313982', '/media/LIN-175%3A+2025-03-04+10%3A27/1_tkz9ulng/365313982', '/media/LIN-175%3A+2025-03-11+10%3A27/1_83t7iz4h/36

In [10]:
list_media_links = list(media_links)
first_media_link_tuple = list_media_links[0]

video_base = "https://aggievideo.canvas.ucdavis.edu"
test_url = video_base + first_media_link_tuple[0]

driver = webdriver.Chrome(options=chrome_options)

for cookie in video_cookies:
  driver.execute_cdp_cmd('Network.setCookie', cookie)

driver.get(test_url)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
print(soup.find("body").prettify())

driver.quit()

<body class="module-default controller-user action-user-media menu-horizontal v2ui shrink--enabled">
 <div id="pageTop">
 </div>
 <div aria-label="Action in progress" aria-live="assertive" id="loaderWrap" role="alert">
 </div>
 <div id="contentWrap">
  <div class="container-fluid" id="wrap">
   <div id="overlay">
    <div id="ajaxStatusWrap">
     <div id="ajaxStatusMain">
     </div>
    </div>
   </div>
   <div id="alerts">
   </div>
   <div class="row-fluid" id="content">
    <div class="row-fluid" id="content">
     <div class="navbar row-fluid">
      <div class="pull-left" id="titleBar">
       <h1 class="inline">
        My Media
       </h1>
      </div>
      <div class="nav pull-right">
       <span class="my-media-user-buttons pull-left">
        <span class="inline hidden-phone hostedEnabled pull-left">
         <a aria-label="Refresh your media library" data-original-title="Refresh your media library" href="/user/refresh-entries" id="refresh" rel="async" role="button" titl

In [23]:
list_media_links = list(media_links)
first_media_link = list_media_links[0]

video_base = "https://aggievideo.canvas.ucdavis.edu"
test_url = video_base + first_media_link

driver = webdriver.Chrome(options=chrome_options)

for cookie in video_cookies:
  driver.execute_cdp_cmd('Network.setCookie', cookie)

driver.get(test_url)


html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
print(soup.find("body").prettify())

tab_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '//ul[contains(@class, "tabs-container")]//a[@id="tab-attachments-tab"]')
))
tab_element.click()
time.sleep(0.5)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
# print(soup.find("body").prettify())
download_pattern = re.compile(r"\.txt$")
anchor = soup.find("a", title=download_pattern)
if anchor:
  link = anchor.get("href")
  title = soup.find("h1", class_="entryTitle")
  print(title.text.strip() if title else "title not found", link)  
else:
  print("anchor not found, exitting")


driver.quit()

<body class="module-default controller-entry action-view menu-horizontal player-v7 v2ui shrink--enabled">
 <div id="pageTop">
 </div>
 <div aria-label="Action in progress" aria-live="assertive" id="loaderWrap" role="alert">
 </div>
 <div id="contentWrap">
  <div class="container-fluid" id="wrap">
   <div id="overlay">
    <div id="ajaxStatusWrap">
     <div id="ajaxStatusMain">
     </div>
    </div>
   </div>
   <div id="alerts">
   </div>
   <div class="row-fluid" id="content">
    <div class="row-fluid" id="content">
     <div class="entry-sub-header span12">
     </div>
     <div class="span9 no-space" id="mediaContainer">
      <div class="video" id="wrapper">
       <!--[if !IE]> -->
       <style>
        #wrapper.video {
            padding-top: 30px !important;
            padding-bottom: 56.25% !important;
        }
       </style>
       <!-- <![endif]-->
       <div id="player">
        <!-- kdp embed -->
        <div class="playerv3" id="kplayer">
         <div class="play